# Food synset entities


### Summary 
105218 food entities

In [1]:
from datetime import datetime
startTime = datetime.now()

In [2]:
%load_ext autotime
%load_ext autoreload
%autoreload 2
import myfunctions as f
f.today()

'20180409'

In [3]:
import sys
print "system_info: %s"%sys.version
# current working directory
import os
print "path_info: %s"%os.getcwd()  

system_info: 2.7.14 |Anaconda custom (64-bit)| (default, Oct 27 2017, 18:21:12) 
[GCC 7.2.0]
path_info: /home/yueliu/Desktop/workspace_yue/Documentation_201712
time: 14.1 ms


** current food synset **

In [4]:
file_name = "food_entities__20180409.p"
df_retrived = f.retrive_file(file_name)

retrived: food_entities__20180409.p
time: 36.8 s


In [5]:
print ("Got %d food records" % df_retrived.shape[0])
print ("Got %d unique food items " % df_retrived["clean_name"].nunique())
print df_retrived.columns.tolist()

Got 143973 food records
Got 143973 unique food items 
['clean_name', 'loc', 'food_name', 'timestamp', 'price', 'vendors', 'tag', 'desc']
time: 55.6 ms


In [6]:
df_retrived.sample(2, random_state=2008)

,clean_name,loc,food_name,timestamp,price,vendors,tag,desc
index,,,,,,,,
food_53819,grill pepper chicken,"[foodpanda/menu_item/10850, foodpanda/menu_ite...","[(W48. Grill Pepper Chicken, 82)]","[2017-07-11 01:13:18.670689, 2017-07-15 00:25:...","(9.63, 9.63, 9.63, 9.63, 9.63, 9.63, 9.63, 9.6...",[delivery_202],[Western],"[nan, Default]"
food_131936,the roots,"[deliveroo/food/26634, deliveroo/food/AV06vmHe...","[(The Roots, 10)]","[2017-07-03T17:55:01.668428, 2017-07-13T14:37:...","(16.05, 16.05, 16.05, 16.05, 16.05, 16.05, 16....","[delivery_6137, delivery_6138]","[Burgers, Burgers ]","[Seasonal vegetables, cheese fritter, coleslaw..."


time: 118 ms


** entity linking by lemmatization **

In [9]:
import spacy
nlp = spacy.load('en_core_web_sm')
def lemma_name(s):
    return " ".join([token.lemma_ for token in nlp(unicode(f.get_string(s), "utf-8"))])

time: 2.05 s


In [ ]:
if 1==1:
    df_retrived["lemma_name"] = df_retrived["clean_name"].apply(lemma_name)
    print ("Got %d unique lemmatized name " % df_retrived["lemma_name"].nunique())

In [ ]:
df_retrived.sample(2, random_state=2008)

** entity linking by sim-hash **

In [ ]:
# warnings: big bucket found, len(dups) > 200, not affect results much
import warnings
warnings.filterwarnings('ignore')
df = f.simhash_name(df_retrived, width=2, k =4, col="lemma_name")
#13min 37s

In [ ]:
print ("Got %d unique name " % df_retrived["name"].nunique())

In [ ]:
df.sample(2, random_state=2008)

** entity linking by removing unecessary words **

In [ ]:
# import file, longer words first
file_name = "words_to_remove_20180308.csv"
import pandas as pd
rmv = pd.read_csv(file_name, header=None)[0].tolist()
rmv.sort(key=lambda item: (-len(item), item))
print "number of words to remove: %d"%len(rmv) 

In [ ]:
def shorten_name(name):
    name = " " + name + " "
    for s in rmv:
        if " " + s + " " in name:
            name = name.replace(" " + s + " ", " ")
    return name.strip()
df["assigned_name"] = df["name"].apply(shorten_name)
df.reset_index(inplace=True)

In [ ]:
print ("Got %d unique short name " % df["assigned_name"].nunique())

In [ ]:
df.sample(2, random_state=2008)

In [ ]:
%%time
# merge records based on name
tup = lambda g: tuple(g)
sort_tup= lambda g: sorted(f.flatten(g), key=lambda x:-x[-1])
unq = lambda g:  sorted(list(set(g)))
unq_enlist = lambda g:  sorted(list(set(f.flatten(g))))
fn = {"food_name":sort_tup, 
      "clean_name":unq, 
      "lemma_name":unq,
      "vendors":unq_enlist,  
      "loc":unq_enlist, 
      "timestamp":unq_enlist, 
      "price":tup,
      "tag":unq_enlist, 
      "desc":unq_enlist,
      "index":tup
     }
df2 = df.groupby("assigned_name").agg(fn)
df2.reset_index(inplace=True)

** select useful columns **

In [ ]:
df2.rename(columns={'clean_name': 'name_variants', 
                     'food_name':"menu_name",
                     'index':'identification_index'}, inplace=True)
cols = ['assigned_name', 'name_variants', 'menu_name',
        'tag', 'price', 'desc','vendors', 'identification_index', 'loc','timestamp', 
        ]
df2 = df2[cols]

In [ ]:
# get proper price
import re
import numpy as np
def eval_price(price):
    return [float(i) for i in re.findall("\d+\.\d+", str(price))]
df2["price"] = df2["price"].apply(eval_price)
def avg_price(lst):
    return round(np.mean(lst),2)
df2["price_average"] = df2["price"].apply(avg_price)

** set index and save file**

In [ ]:
df2.reset_index(inplace=True)
df2["index"] = "food_item_" + df2["index"].apply(str)
df2.set_index("index", inplace=True)

In [ ]:
print df2.shape
print df2.columns.tolist()
df2.sample(2,random_state=2008*2)

In [ ]:
## save to p file
file_name = "__".join(["food_entities", f.today()]) + ".p"
f.save_file(df2, file_name)

** public version **

In [ ]:
public_cols = ['assigned_name', 'name_variants', 'price_average']
## save to json
file_name = "__".join(["public_food_entities", f.today()]) + ".json"
df2[public_cols].to_json(file_name, orient='index')
print "saved: %s"%file_name

In [ ]:
# total time
print datetime.now() - startTime 